In [ ]:
!pip install pyspark findspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Создание DataFrame**

Из RDD

In [ ]:
def cleaning(row):
    row = row.split('\t')[:3]
    row = [float(val) for val in row]
    return row

In [ ]:
ratings = sc.textFile('user_ratedmovies.dat')

first_row = ratings.first()
ratings = ratings.filter(lambda row: row != first_row)\
                 .map(cleaning)

In [ ]:
columns = first_row.split('\t')[:3]

In [ ]:
columns

In [ ]:
ratings.take(5)

In [ ]:
df_rdd = spark.createDataFrame(ratings, columns)

In [ ]:
df_rdd

Можно еще вот так:

In [ ]:
df_rdd = ratings.toDF(columns)

In [ ]:
df_rdd

Так, а если не хочу вот эти приседания с RDD, а хочу сразу из файла?

In [ ]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .load("user_ratedmovies.dat")

In [ ]:
df

Все в string, так не пойдет, давайте автоматически определим тип данных

In [ ]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true', 'inferSchema': 'true'})\
          .load("user_ratedmovies.dat")

In [ ]:
df

In [ ]:
df.show(10)

А можно заранее сказать, какой тип данных я ожидаю?

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [ ]:
schema = StructType([ \
    StructField("userID",IntegerType(),True), \
    StructField("movieID",IntegerType(),True), \
    StructField("rating",DoubleType(),True), \
    StructField("date_day", StringType(), True), \
    StructField("date_month", StringType(), True), \
    StructField("date_year", IntegerType(), True), \
    StructField("date_hour", IntegerType(), True), \
    StructField("date_minute", IntegerType(), True), \
    StructField("date_second", IntegerType(), True)
  ])

In [ ]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .schema(schema)\
          .load("user_ratedmovies.dat")

In [ ]:
df.printSchema()

In [ ]:
df.show(10)

Но есть уже готовая обертка под все нужды

In [ ]:
df = spark.read.csv(path='user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [ ]:
df.printSchema()

Так, а как сохранить? Лучше быть аккуратнее с overwrite, перезапишет весь указанный путь, append будет безопаснее

In [ ]:
df.write.option("header",True)\
        .mode("overwrite")\
        .parquet('write_1.parquet')

А что с партицированием?

In [ ]:
df.write.option("header",True)\
        .partitionBy('date_year')\
        .mode("overwrite")\
        .parquet('write_2.parquet')

Кстати, раз уж заговорили про схемы данных, то их можно задавать интереснее, например, под группированные данные

In [ ]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

Со структурой можно работать и менять ее под ваши нужны

In [ ]:
updatedDF = df2.withColumn("OtherInfo", 
    F.struct(F.col("id").alias("identifier"),
    F.col("gender").alias("gender"),
    F.col("salary").alias("salary"),
    F.when(F.col("salary").cast(IntegerType()) < 2000,"Low")
      .when(F.col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

Что мы там сделали????

1) Создали новую структуру данных OtherInfo

2) Передали туда id (переименовав столбец), gender, salary

3) Создали столбец Salary_grade из условий

4) удалили id, gender, salary из старой структуры

Есть и еще структуры данных!

In [ ]:
from pyspark.sql.types import ArrayType, MapType

In [ ]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

In [ ]:
structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

In [ ]:
df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

In [ ]:
df3.select('properties').collect()

**Описание данных**

Общее описание данных

In [ ]:
df = spark.read.csv(path='user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [ ]:
df.describe().show()

In [ ]:
df.summary().show()

Количество записей

In [ ]:
df.count()

Количество партиций

In [ ]:
df.rdd.getNumPartitions()

 Менять число партиций можно, все как с rdd

In [ ]:
df = df.repartition(5)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df = df.coalesce(2)

In [ ]:
df.rdd.getNumPartitions()

**Различные методы**

Ну теперь давайте тыкать 

Удаляем дубликаты и помним, что есть actions и transformations, count заставит все сделать

In [ ]:
df_without_duplicates = df.drop_duplicates()

Есть alias

In [ ]:
df_without_duplicates = df.dropDuplicates()

Как удалить дубликаты по отдельным колонкам?

In [ ]:
df_without_duplicates = df.drop_duplicates(['userID', 'rating'])

In [ ]:
df_without_duplicates.count()

In [ ]:
df_without_duplicates.show(10)

Корреляции

In [ ]:
df.corr('rating', 'date_day')

In [ ]:
df.corr('rating', 'date_hour')

In [ ]:
df.corr('rating', 'date_year')

Как закинуть данные в любимый pandas?

 Самый простой вариант - встроенный метод, но он жутко медленный при существенном объеме данных. Лучше сохранить паркет и считать через pd.read_parquet()

In [ ]:
import pandas as pd

In [ ]:
%%timeit 

pandas_df = df.toPandas()

In [ ]:
pandas_df = df.toPandas()
pandas_df

In [ ]:
pandas_df.info()

In [ ]:
%%timeit 

pandas_df = pd.read_parquet('write_1.parquet')

In [ ]:
pandas_df = pd.read_parquet('write_1.parquet')
pandas_df

In [ ]:
pandas_df.info()

Как говорили на лекции, может все упасть например тут (когда делаем toPandas). Как перейти к итератору?

prefetchPartitions - подготавливать ли следующую партию данных, пока не запросили

In [ ]:
iter_df = df.toLocalIterator()

In [ ]:
row = iter_df.send(None)

In [ ]:
row

In [ ]:
row.asDict()

Отсюда идея: можно вытягивать данные по 1 записи и записывать в датафрейм. Долго, но зато отработает.

In [ ]:
iter_df = df.toLocalIterator()

In [ ]:
list_of_rows = [value for value in iter_df]
print(len(list_of_rows))

In [ ]:
df.columns

In [ ]:
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

In [ ]:
%%timeit

iter_df = df.toLocalIterator()
list_of_rows = [value for value in iter_df]
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

In [ ]:
pandas_df

In [ ]:
pandas_df.info()

**Show**

In [ ]:
df.show(10)

Обрезаем до 2 символов

In [ ]:
df.show(10, truncate=2)

вертикальное отображение

In [ ]:
df.show(10, vertical=True)

**Select**

В PySpark функция select() используется для выбора одного, нескольких столбцов по индексу, всех столбцов из списка и вложенных столбцов из фрейма данных. Функция PySpark select() является функцией преобразования, поэтому она возвращает новый фрейм данных с выбранными столбцами.

In [ ]:
df.columns

Упс, pandas-style тут не приветствуется

In [ ]:
df.userID.show(5)

In [ ]:
df.select('userID').show(5)

Куча вариантов, выбирайте любой

In [ ]:
df.select('userID', 'rating').show(5)

In [ ]:
df.select(['userID', 'rating']).show(5)

In [ ]:
df.select(df.userID,df.rating).show(5)

In [ ]:
df.select(df['userID'],df['rating']).show(5)

In [ ]:
df.select(F.col("userID"), F.col("rating")).show(5)

можно налету переименовать столбец

In [ ]:
df.select(df.userID, df.rating.alias('mark')).show(5)

In [ ]:
#регулярки
df.select(df.colRegex("`d+.*y`")).show(5)

примеры с другим датафреймом, где структура сложнее

In [ ]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

In [ ]:
df2.select('name').show(5)

In [ ]:
df2.select('name.lastname').show(5)

In [ ]:
df2.select('name.firstname', 'name.lastname').show(5)

**withColumn**

PySpark withColumn() - это функция преобразования (transform), которая используется для изменения значения, преобразования типа данных существующего столбца, создания нового столбца и многого другого. Поговорим о часто используемых операциях со столбцами данных PySpark, используя примеры.

In [ ]:
df.printSchema()

Меняем тип данных

In [ ]:
df.withColumn("date_month", F.col("date_month").cast("String")).printSchema()

Модифицировать столбец/создать новый

In [ ]:
df.withColumn("rating_x_10",F.col("rating") * 10).show(5)

Делаем 2 константных столбца

In [ ]:
df.withColumn('fix_1', F.lit(1)).withColumn('fix_2', F.lit(2)).show(5)

**withColumnsRenamed**

Предыдущий вариант не давал возможности переименовать столбцы, это можно сделать иначе

In [ ]:
df.withColumnRenamed('rating', 'mark').show(5)

**filter (where) и иные филтрации**

Функция PySpark filter() используется для фильтрации строк из RDD / DataFrame на основе заданного условия или выражения SQL, вы также можете использовать предложение where() вместо filter() обе эти функции работают аналогично.

1 условие

In [ ]:
df.filter(df.rating == 5.0).show(5)

In [ ]:
df.filter(~(df.rating == 5.0)).show(5)

In [ ]:
df.filter('rating = 5').show(5)

Несколько условий

In [ ]:
df.filter((df.rating == 5.0) & (df.date_year == 2006)).show(5)

In [ ]:
df.filter('(rating = 5.0) and (date_year = 2006)').show(5)

In [ ]:
df.filter('(rating = 5.0) and (userID between 70 and 80)').show(5)

фильтр по списку значений из list

In [ ]:
years = [2006, 2007]

In [ ]:
df.filter(df.date_year.isin(years)).show(5)

 проверим

In [ ]:
df.filter(df.date_year.isin(years))\
  .select('date_year')\
  .dropDuplicates()\
  .collect()

создадим игрушечный датайфрейм для текстовых столбцов

In [ ]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5,"Rames Black"), (6, 'Albus Torch'),
     (7, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

In [ ]:
df2.show()

In [ ]:
df2.filter('name like "R%"').show()

In [ ]:
df2.filter(df2.name.startswith('R')).show()

In [ ]:
df2.filter(df2.name.endswith('Tf')).show()

In [ ]:
df2.filter(df2.name.contains('Wil')).show()

Бывает, что у нас внутри датафрейма есть массив и с ним что-то хочется сделать

In [ ]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

In [ ]:
df3.filter(F.array_contains(df3.hobbies,"football")) \
    .show()  

**Сортировка**

сделаем еще фильтрацию, чтобы увидеть результат (orderBy тут аналог)

In [ ]:
df.filter(df.userID == 75).sort(df.date_minute, df.rating.desc()).show(20)

**groupby**

Когда мы выполняем groupBy() в PySpark DataFrame, он возвращает объект GroupedData, который содержит следующие агрегатные функции:

min(), max(), mean(), count(), sum(), avg(), agg(), pivot() 

In [ ]:
df.groupby('date_year').mean('rating').collect()

мы уже умеем применять разные методы

In [ ]:
df.groupby('date_year')\
  .mean('rating')\
  .sort('date_year')\
  .collect()

In [ ]:
df.filter(df.rating <= 2)\
  .groupby('date_year')\
  .count()\
  .withColumnRenamed('count', 'number')\
  .sort('date_year')\
  .collect()

несколько колонок

In [ ]:
df.groupBy("date_year", "date_month") \
  .mean("rating", "userID") \
  .sort('date_year', 'date_month') \
  .show()

Для того, чтобы делать несколько разных агрегаций и еще менять сразу имя столбца нужно немного изменить синтаксис

И стоит помнить, что персентили тут считаются приближенно

In [ ]:
df.groupBy("date_year") \
    .agg(F.min("rating").alias("min_rating"), \
         F.mean("rating").alias("mean_rating"), \
         F.max("rating").alias("max_rating"),
         F.percentile_approx("rating", 0.5).alias("median")
         ) \
    .show()

Еще можно сделать pivot

In [ ]:
df.groupBy('date_year')\
  .pivot('date_month')\
  .mean('rating')\
  .show(5)

**Join's**

Куда же без них. Что есть: INNER, LEFT OUTER, RIGHT OUTER, LEFT ANTI, LEFT SEMI, CROSS, SELF JOIN

Благодаря оптимизации в датафреймах уже все хорошо работает, спасибо catalist, но чудеса не вечны и плохой код/незнание данных все равно даст о себе знать

Сделаем для себя несколько таблиц, чтобы можно было экспериментировать

In [ ]:
df_mean_user_rating = df.groupBy("userID")\
                        .mean('rating')\
                        .withColumnRenamed('avg(rating)', 'avg_rating_all')

df_mean_user_rating_year = df.groupby('userID', 'date_year')\
                             .mean('rating')\
                             .withColumnRenamed('avg(rating)', 'avg_rating_year')

In [ ]:
df_mean_user_rating.printSchema()

df_mean_user_rating_year.printSchema()

И давайте все в 1 блоке кода, чтобы не растягивать

In [ ]:
df.join(df_mean_user_rating, on=df.userID == df_mean_user_rating.userID, how='inner')\
  .join(df_mean_user_rating_year, on=[df.userID == df_mean_user_rating_year.userID,
                                      df.date_year == df_mean_user_rating_year.date_year],
        how='inner')

Надо удалить дублирующиеся столбцы

In [ ]:
res_join = df.alias('t').join(df_mean_user_rating.alias('t1'), on=F.col('t.userID') == F.col('t1.userID'), how='inner')\
  .drop(F.col('t1.userID'))\
  .join(df_mean_user_rating_year.alias('t2'), on=[F.col('t.userID') == F.col('t2.userID'),
                                      F.col('t.date_year') == F.col('t2.date_year')],
        how='inner')\
  .drop(F.col('t2.userID'))\
  .drop(F.col('t2.date_year'))

In [ ]:
res_join.show(5)

**union и unionAll**

Используются для объединения датафреймов с одинаковой структурой, используется union, так как unionAll с версии 2.0.0 более не используется

In [ ]:
df1 = df.filter(df.date_year == 2006)
df2 = df.filter(df.date_year != 2006)

In [ ]:
union_df = df1.union(df2)

print(df.count(), union_df.count())

Desclaimer: все по sql, надеюсь, помнят разницу между union и union all, когда union убирает дубликаты. Так вот pyspark ничего не удаляет, убрать дубликаты можно только через drop_duplicates, distinct

Также при union pyspark делает объединение по столбцам as is, не пытаясь понять, что в одном датафрейме нужный стобец на 1 позиции, а в другом он на 5. Для этого с версии 3.1 есть замечательный метод unionByName

**UDF - user defined functions**

из курса про rdd помним про map, тут тоже можно перегнать все в rdd и делать map, но можно и через udf. Стоит отметить, что при этом мы теряем возможность оптимизации и произодительность в dataframe, так как udf - black box для спарка.

Но зато эти функции переиспользуемы и их можно применять в sql запросах, как те же udf в oracle

In [ ]:
def udf_example(rating):
    rating = rating * 20
    return rating

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
my_udf = udf(lambda x: udf_example(x), DoubleType())

In [ ]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', my_udf(F.col('rating')))\
  .show(5)

Для тех, кто любит декораторы

In [ ]:
@udf(returnType=DoubleType()) 
def udf_example_decorator(rating):
    rating = rating * 20
    return rating

In [ ]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', udf_example_decorator(F.col('rating')))\
  .show(5)

Зарегистрируем функцию для будущих примеров с sql

In [ ]:
spark.udf.register("udf_example_decorator", udf_example_decorator)

**SQL**

Ну раз уж пошла такая тема, давайте рассмотрим, как можно сделать все при помощи любимого SQL

можно делать TempView и GlodalTempView, отличие в том, что обычный view будет жить, пока жива сессия спрака, а глобальная, пока жив sparkcontext

In [ ]:
df.createOrReplaceTempView('df')

In [ ]:
query = '''
select userID, movieID, rating, udf_example_decorator(rating) as rating_100
from
df
'''
spark.sql(query).show(5)

Ну и наш join

In [ ]:
df_mean_user_rating.createOrReplaceTempView('df_mean_user_rating')
df_mean_user_rating_year.createOrReplaceTempView('df_mean_user_rating_year')

In [ ]:
query = '''
select t.*, t1.avg_rating_all, t2.avg_rating_year
from
df t, df_mean_user_rating t1, df_mean_user_rating_year t2
where
    t.userID = t1.userID and
    t.userID = t2.userID and
    t.date_year = t2.date_year
'''
spark.sql(query).show(5)

**fill() и fillna()**

Оба метода идентичны, заполняют пропуски

In [ ]:
import numpy as np

In [ ]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5, None), (6, None),
     (None, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

А где пропуски?

In [ ]:
df2.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

In [ ]:
df2.show()

In [ ]:
df2.fillna({'id': 0}).show()

In [ ]:
df2.fillna({'id': 0, 'name': 'Unknown'}).show()

Аналогично

In [ ]:
df2.na.fill({'id': 0, 'name': 'Unknown'}).show()

Немного поговорим о том, как делаеть подвыборки

**sample и sampleBy**

Не забываем про возможную некоторую недетерминированность

In [ ]:
import pyspark.sql.functions as F

In [ ]:
trans_data = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/spark_transactions.parquet')
trans_data = trans_data.withColumn('target', F.when(F.col('IsFraud')=='No', 0).otherwise(1))

In [ ]:
trans_data.count()

In [ ]:
trans_data.select(F.mean(F.col('target'))).show()

In [ ]:
trans_data.select('target')\
          .groupBy('target')\
          .count()\
          .show()

In [ ]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.1, seed=3)
print(trans_data_simple.count())
trans_data_simple.select(F.mean(F.col('target'))).show()

Стратификация

Тут важно понимать, что это не scikit-learn и стратификация предполагает, что вы по какому-то полю можете выбрать определенную долю наблюдений по его значениям

In [ ]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

Не указали какой-то ключ - его доля будет 0

In [ ]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0, 0: 0.1}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

Оконные функции **F.func().over(Window.partitionBy().orderBy())**

In [ ]:
from pyspark.sql.window import Window

In [ ]:
trans_data_simple.select('User', 'Card', 'Year', 'Month', 'Day')\
.withColumn('rn_1', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc())))\
.withColumn('rn_2', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc(), F.col('Month').desc())))\
.withColumn('mean', F.mean('Day').over(Window.partitionBy('User', 'Card', 'Month')))\
.withColumn('lag', F.lag('Day').over(Window.partitionBy('User', 'Card', 'Month').orderBy(F.col('Year').asc(),
                                                                                         F.col('Month').desc(),
                                                                                         F.col('Day').desc())))\
.show()

Сборка последовательностей через **collect_list()**

In [ ]:
trans_data.printSchema()

In [ ]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Amount', 'Day').alias('sequence'))

In [ ]:
trans_data_simple_seq.show()

In [ ]:
trans_data_simple_seq = trans_data_simple_seq.groupBy('User', 'Card')\
                                             .agg(F.collect_list('sequence').alias('sequence'))

In [ ]:
trans_data_simple_seq.show(truncate=50)

А что с этим делать дальше? Можно перейти к RDD и построчно обрабатывать

In [ ]:
row = trans_data_simple_seq.rdd.take(1)

In [ ]:
row

In [ ]:
row[0]['sequence']

**Домашнее задание**

Куда же без домашки, верно?

Есть данные по транзакциям клиентов, ваша задача состоит в анализе этих данных и подготовки к структуре, которая похожа на ту структуру, которая сейчас часто нами используется при построении моделей на транзакциях + промежуточные задания.

Не забудьте делать всякие show после каждого задания, чтобы было видно результат

**Файл spark_transactions.parquet можете забрать в папке с записями лекций**

**Важно**
В домашнем задании старайтесь использовать максимально dataframe api, а не sql запросы.

In [ ]:
trans_data = spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/spark_transactions.parquet')

In [ ]:
trans_data.count()

In [ ]:
trans_data.show(5)

Посмотрим на схему данных

Сколько в среднем транзакций у пользователя

Сколько карт у пользователей в среднем

Немного обработаем данные: Amount в float, из Time вытянем час транзакции и удалим исходный Time, Zip  к типу int

Посчитайте количество транзакций по годам, учитывая только те транзакции, объем которых был больше 100

Определите, есть ли пропуски в данных по каждому столбцу

Заполните пропуски исходя из типа данных

При помощи оконных функций для каждого клиента рассчитайте средний размер транзакции, количество транзакций и последнюю по дате транзакцию.

Теперь самое время сгруппировать данные по каждому клиенту (можно использовать collect_list для сбора данных после агрегации)
Когда будете делать агрегацию, то возьмите только часть выборки, например, через sample, для всей выборки либо не хватит памяти, либо очень долго считать

Напишите python функцию, которая возьмет данные после агрегации последовательностей, отсортирует их внутри по дате и времени и преобразует к формату python dict:
{'User': User,
'Card': Card,
'sequence':{
    'amount': [последовательность],
    'year': [последовательность],
    'month': [последовательность],
    'day': [последовательность],
    'time': [последовательность],
    'MCC': [последовательность]
}
}




Выведите как пример одну преобразованную запись, результаты сохраните на диск в через rdd pickle